In [2]:
!pip install requests
!pip install bs4
!pip install crewai
!pip install langchain_openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import requests
from bs4 import BeautifulSoup
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [7]:
from google.colab import userdata
key=userdata.get('key')

In [10]:
from googlesearch import search

In [9]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = key

# Define the LLM model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=8000)

In [12]:
class IndustryResearch(Agent):
    def execute(self, input_data):
        """
        Performs industry research and gathers insights about the company's sector and focus areas.
        """
        company = input_data.get("company", "")
        industry_insights = {}

        try:
            # Example: Fetch data using a web API or scraper (Replace with an actual API)
            response = requests.get(f"https://api.serper.dev/search?q={company}+industry+focus")
            data = response.json()

            # Process results
            industry_insights = {
                "industry": data.get("industry", "Unknown"),
                "focus_areas": data.get("focus_areas", [])
            }
        except Exception as e:
            self.log_error(f"Error in IndustryResearchAgent: {e}")
            industry_insights = {"industry": "Generic", "focus_areas": ["General Insights"]}

        return industry_insights

In [13]:
class MarketTrendsAgent(Agent):
    def execute(self, industry_data):
        """
        Fetches AI/ML trends and standards for the given industry.
        """
        industry = industry_data.get("industry", "Unknown")
        focus_areas = industry_data.get("focus_areas", [])
        trends = []

        try:
            # Simulating trend search (Replace with actual search or API call)
            if industry == "Automotive":
                trends = ["Predictive Maintenance", "Autonomous Vehicles", "AI-Driven Quality Control"]
            elif industry == "Healthcare":
                trends = ["AI for Diagnostics", "Telemedicine", "Patient Data Analytics"]
            else:
                trends = ["Generic AI/ML Trends"]

        except Exception as e:
            self.log_error(f"Error in MarketTrendsAgent: {e}")
            trends = ["Fallback AI/ML Trends"]

        return {"industry": industry, "trends": trends}

In [14]:
class UseCaseAgent(Agent):
    def execute(self, trends_data):
        """
        Generates relevant use cases based on trends and focus areas.
        """
        trends = trends_data.get("trends", [])
        use_cases = []

        try:
            for trend in trends:
                if "Predictive Maintenance" in trend:
                    use_cases.append("AI models for equipment failure prediction.")
                elif "Autonomous Vehicles" in trend:
                    use_cases.append("AI for self-driving technology and fleet management.")
                elif "Quality Control" in trend:
                    use_cases.append("Computer vision for automated defect detection.")
                else:
                    use_cases.append(f"AI use case related to {trend.lower()}.")

        except Exception as e:
            self.log_error(f"Error in UseCaseAgent: {e}")
            use_cases = ["Generic AI use cases"]

        return {"use_cases": use_cases}

In [15]:
class ResourceCollectionAgent(Agent):
    def execute(self, use_case_data):
        """
        Finds relevant datasets and tools for the use cases.
        """
        use_cases = use_case_data.get("use_cases", [])
        resources = []

        try:
            for use_case in use_cases:
                # Example: Searching datasets via Kaggle API or web scraping
                query = use_case.split(" ")[0]  # Basic keyword extraction
                response = requests.get(f"https://www.kaggle.com/search?q={query}")

                # Simulated response processing
                resources.append({
                    "use_case": use_case,
                    "dataset_name": f"Sample Dataset for {query}",
                    "link": f"https://kaggle.com/sample-dataset-{query}"
                })

        except Exception as e:
            self.log_error(f"Error in ResourceCollectionAgent: {e}")
            resources.append({"use_case": "Fallback", "dataset_name": "Generic Dataset", "link": "https://kaggle.com"})

        return {"resources": resources}

In [16]:
class EditorAgent(Agent):
    def execute(self, industry_data, trends_data, use_cases_data, resources_data):
        """
        Compiles all the outputs into a markdown file.
        """
        industry = industry_data.get("industry", "Unknown")
        focus_areas = ", ".join(industry_data.get("focus_areas", []))
        trends = "\n".join([f"- {trend}" for trend in trends_data.get("trends", [])])
        use_cases = "\n".join([f"- {use_case}" for use_case in use_cases_data.get("use_cases", [])])
        resources = "\n".join([
            f"- **{res['use_case']}**: [{res['dataset_name']}]({res['link']})"
            for res in resources_data.get("resources", [])
        ])

        # Create the markdown content
        markdown_content = f"""
# Final Proposal: AI/GenAI Use Cases for {industry}

## Industry Insights
- **Industry**: {industry}
- **Focus Areas**: {focus_areas}

## Market Trends & Standards
{trends}

## Proposed Use Cases
{use_cases}

## Resource Assets
{resources}

---
*Generated by the Multi-Agent System using CrewAI.*
"""
        # Save to a markdown file
        with open("final_proposal.md", "w") as file:
            file.write(markdown_content)

        return {"status": "success", "file": "final_proposal.md"}

In [ ]:
from crewai.crew import Crew
class MultiAgentCrew(Crew):
    def execute(self, input_data):
        """
        Orchestrates the agents in sequence.
        """
        # Step 1: Research the industry
        industry_agent = IndustryResearchAgent()
        industry_data = industry_agent.execute(input_data)

        # Step 2: Fetch market trends
        trends_agent = MarketTrendsAgent()
        trends_data = trends_agent.execute(industry_data)

        # Step 3: Generate use cases
        use_case_agent = UseCaseAgent()
        use_cases_data = use_case_agent.execute(trends_data)

        # Step 4: Collect resource assets
        resource_agent = ResourceCollectionAgent()
        resources_data = resource_agent.execute(use_cases_data)

        # Step 5: Compile final proposal
        editor_agent = EditorAgent()
        proposal = editor_agent.execute(industry_data, trends_data, use_cases_data, resources_data)

        return proposal
